In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training
from methods_audio import model_training

# In the following code, the best model (resulting from experiments) will be trained once and exported as final model. 

1. Get data

In [ ]:
data = data_handling.get_data()

2. Read data (transforming file names into waves)

In [ ]:
data = data.map(data_handling.read_in_data) 

3. Get input for model training 

In [ ]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-06 18:51:25.140497: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4. Split data into train and validation sets

In [ ]:
validation_set_size = 0.30
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size= validation_set_size, random_state=123)

Set parameters

In [ ]:
# augmentation 
type_augmentation = 'signal'
# denoising 
type_denoising= 'spectral'
differentiation = True 

# Low pass filter 
low_pass_cutoff = 500 # this value will be used for experimentation, ranging from 500-4000. 
low_pass_order = 4 # this value will be used for experimentation 

# transforming data
type_transformation = 'spectrogram' 

# model structure 
model_name = '01'
location_model = 'data/models/' + model_name 
location_weights = 'data/models/weights' + model_name

# training 
batch_size = 8 
epoch = 20 

# learning rate 
if (model_name == '01'):
    learning_rate = 0.02661877777328162 # result from param optimization for Model 1 
elif (model_name == '02'): 
    learning_rate = 0.01

5. Data augmentation

In [ ]:
if (type_augmentation == 'signal'):
    x_train, y_train = data_augmentation.time_augmentation(x_train, y_train)
elif(type_augmentation == 'spectrogram'):
    # TODO: work out what to do  
    x_train, y_train = data_augmentation.spectrogram_augmentaion(x_train, y_train)
elif(type_augmentation == 'both'): 
    # TODO: sort this
    pass 

5. Denoising 

In [ ]:
if (type_denoising== 'spectral'): 
    denoising.apply_spectral(x_train)
    denoising.apply_spectral(x_valid)
elif(type_denoising == 'low_pass'): 
    denoising.apply_low_pass(x_train, low_pass_cutoff, low_pass_order)
    denoising.apply_low_pass(x_valid, low_pass_cutoff, low_pass_order)

7. Data transformation 

In [ ]:
x_train = data_handling.transform_data(x_train, type_transformation)
x_valid = data_handling.transform_data(x_valid, type_transformation)

8. Upload and compile model 

In [ ]:
model = keras.models.load_model(location_model)
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=learning_rate), 
    loss="BinaryCrossentropy", 
    metrics = ['accuracy', 'Recall', 'Precision'],
    )

9. Train model 

In [ ]:
model, history = model_training(model, x_train, y_train, x_valid, y_train, batch_size, epoch)


10. Save weights 

In [ ]:
model.save_weights(location_weights, overwrite = True, save_format = 'tf', options = None) # saving as TensorFlow format 